# コードのプロファイリングと高速化

In [1]:
import math
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import *
from tqdm import tqdm_notebook as tqdm
from decimal import *
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

# 因数検索関数の高速化

In [2]:
Num = 10**4
%timeit nlist=[k for k in range(2,Num+1) if k%3 == 0]
%timeit num = 5; nlist = [num*k for k in range(1,int(Num/num)+1)]

1000 loops, best of 3: 494 µs per loop
10000 loops, best of 3: 94.7 µs per loop


In [7]:
def find_factor_01(num):
    factors = []
    mx = int(math.sqrt(num))    
    for k in range(2, mx+1):
        if num%k==0: factors.append(k)
    return factors
            
def find_factor_02(num):
    factors = []
    mx = int(math.sqrt(num))
    composites = []
    for k in range(2, mx+1):
        if k not in composites:
            if num%k==0: factors.append(k)
            else: composites.extend([k*i for i in range(2, int(mx/k)+1)])
    return factors
                
def find_factor_03(num):
    factors = []
    nlist = list(range(2, int(math.sqrt(num))+1))
    while nlist:
        mm = nlist.pop(0)
        res = num%mm
        if res == 0:
            factors.append(mm)
            break
        else:
            nlist = [k for k in nlist if k%mm > 0]
    return factors

# プロセス所要時間計測

In [17]:
num = 2**23-1
%timeit find_factor_01(num)
%timeit find_factor_02(num)
%timeit find_factor_03(num)

1000 loops, best of 3: 154 µs per loop
10 loops, best of 3: 63 ms per loop
1000 loops, best of 3: 566 µs per loop


# プロファイリング

In [4]:
%load_ext line_profiler

In [8]:
num = 2**31-1
%lprun -f find_factor_01 find_factor_01(num)
#%lprun -f find_factor_01 find_factor_01(num)
#%lprun -f find_factor_01 find_factor_01(num)

### find_factor_01 のプロファイリング結果
```
Total time: 0.062922 s
File: <ipython-input-3-799ddd921f51>
Function: find_factor_01 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def find_factor_01(num):
     2         1         12.0     12.0      0.0      mx = int(math.sqrt(num))    
     3     46340      27042.0      0.6     43.0      for k in range(2, mx+1):
     4     46339      35868.0      0.8     57.0          if num%k==0: print(k)
```

### find_factor_02 のプロファイリング結果
```
Total time: 16.8452 s
File: <ipython-input-6-02803966a958>
Function: find_factor_02 at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     8                                           def find_factor_02(num):
     9         1          8.0      8.0      0.0      mx = int(math.sqrt(num))
    10         1          2.0      2.0      0.0      composites = []
    11     46340      38539.0      0.8      0.2      for k in range(2, mx+1):
    12     46339   16744590.0    361.3     99.4          if k not in composites:
    13      4792       8903.0      1.9      0.1              if num%k==0: print(k)
    14      4792      53194.0     11.1      0.3              else: composites.extend([k*i for i in range(2, int(mx/k)+1)])
```

### find_factor_03 のプロファイリング結果
```
Total time: 2.02016 s
File: <ipython-input-6-02803966a958>
Function: find_factor_03 at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    16                                           def find_factor_03(num):
    17         1       1999.0   1999.0      0.1      nlist = list(range(2, int(math.sqrt(num))+1))
    18      4793       4116.0      0.9      0.2      while nlist:
    19      4792       9775.0      2.0      0.5          mm = nlist.pop(0)
    20      4792       3365.0      0.7      0.2          res = num%mm
    21      4792       2979.0      0.6      0.1          if res == 0:
    22                                                       print(mm)
    23                                                       break
    24                                                   else:
    25      4792    1997931.0    416.9     98.9              nlist = [k for k in nlist if k%mm > 0]
```